In [1]:
import sys, os, time
os.environ["CUDA_VISIBLE_DEVICES"]="0"
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distrib

import torch.multiprocessing as mp
import torchvision.models
from torch.utils.data import Dataset, DataLoader
import gym
import numpy as np
import h5py as hf
# %matplotlib notebook
%matplotlib tk
import matplotlib.pyplot as plt
#plt.switch_backend('Qt5Agg') #('Qt5Agg')
import foundation as fd
from foundation import models
from foundation import util
from foundation import train
from foundation import sim as SIM
#from foundation.util import replicate, Cloner

np.set_printoptions(linewidth=120, suppress=True)

In [2]:
fig = None
figE, axsE = plt.subplots(2)

In [3]:
auto_lim = None
auto_lim = 1.
sim = SIM.Cartpole(timestep=0.02, ctrl_scale=5,
                   length=.5, gravity=10., 
                   mass_cart=1, mass_pole=1,
#                    length=1, gravity=1., 
#                    mass_cart=1, mass_pole=1,
                   fric_cart=0.1, fric_pole=0.2,
                   integration_method='rk4',
                   limit=auto_lim, limit_pow=3., limit_coeff=-40.,
                  )
sim

Cartpole(
  (dynamics): Cartpole_Dynamics(
    (actuator): Composer(
      (controllers): ModuleList(
        (0): Constant()
        (1): SimpleLimiter(
          (force): CutPowerForce()
        )
      )
    )
  )
  (controller): Constant()
)

In [4]:
sim.dynamics.friction_full

True

In [5]:
# sim.sample_state()
# sim.sample_action()

In [6]:
B = None
state0 = None
# state0 = torch.tensor([0.,np.pi,0,0])
state0 = torch.tensor([0.,np.pi,6,0])
# state0 = sim.sample_state(B)

In [7]:
proc = util.OUNoise(batch_size=B, mu=0., sigma=0.4, theta=.2)

In [8]:
if fig is None:
    if B is not None and int(np.sqrt(B))**2 == B:
        bb = int(np.sqrt(B))
        fig, axes = plt.subplots(bb,bb)
    else:
        fig, ax = plt.subplots(1)

In [9]:

limit = 1
k = 40.
m = 4
    
def lim_fn(self, x, theta, dx, dtheta):
    return -k*x**(m-1)#*(dx.abs()**4+0.9)
def lim_E(self, x, theta, dx, dtheta):
    return k/m*x**m

def add_lim_E(self, x, theta, dx, dtheta, V):
    sel = (x.abs()>limit)
    
#     return V
    
    V[sel] += lim_E(self, x[sel], theta, dx, dtheta)
    
#     V[sel] += lim_E(x[sel])
    
    return V

def full_lim(self, x, theta, dx, dtheta):
    sel = (x.abs()>limit)
    
    lim_x, lim_theta = torch.zeros_like(x), torch.zeros_like(theta)
    
    x, theta, dx = x[sel], theta[sel], dx[sel]
    
    lim_x[sel] = lim_fn(self, x, theta, dx, dtheta)
    lim_theta[sel] = lim_x[sel]#*torch.cos(theta)
    
#     print(lim_x)
    
    return lim_x, lim_theta
    
def full_lim2(self, x, theta, dx, dtheta):
    sel = (x.abs()>limit)
    
    lim = torch.zeros_like(x)
    
    x, theta, dx = x[sel], theta[sel], dx[sel]
    
    lim[sel] = lim_fn(self, x, theta, dx, dtheta)
    
#     print(lim)
    
    return lim
    
sim.dynamics.lim_fn = full_lim
sim.dynamics.lim_V = add_lim_E
sim.dynamics.new_lim = full_lim2
sim.dynamics.old_dynamics = False
sim.dynamics.use_lim = auto_lim is None

In [10]:
sim.dynamics.mass_pole, sim.dynamics.mass_cart, sim.dynamics.length, sim.dynamics.gravity, k, m

(1, 1, 0.5, 10.0, 40.0, 4)

In [11]:
def get_action(i, x):
    return -torch.ones(1)*0
    return proc()

In [15]:
x = sim.reset(state0)
Es = []
Ts, Vs = [], []
traj = []
# x = sim.reset()
render = True

In [13]:
render ^= True
render

False

In [16]:
plt.figure(fig.number)
for i in range(200):
    u = sim.sample_action(B)
    u = get_action(i,x)
    x = sim.step(u, N=1)
    traj.append(x.numpy())
    E = sim.dynamics.energy(x).item()
    Ts.append(sim.dynamics.kinetic_energy(x).item())
    Vs.append(sim.dynamics.potential_energy(x).item())
    Es.append(E)
    if render:
        imgs = sim.render(200,100)
        if len(imgs.shape) == 4:
            for img, ax in zip(imgs, axes.flat):
                ax.cla()
                ax.imshow(img)
                ax.axis('off')
        else:
            ax.cla()
            ax.imshow(imgs)
    #         ax.axis('off')
            plt.xticks([])
            plt.yticks([])
            plt.xlabel('{:2.3f} {:2.3f}'.format(E, u.item()))
        plt.pause(0.03)

plt.sca(axsE[0])
plt.cla()
plt.plot(Es,label='H')
plt.plot(Ts,label='T')
plt.plot(Vs,label='V')
# plt.ylim(-0.01,Es[0]*2+0.01)
# plt.ylim(-0.01,10)
plt.legend()

plt.sca(axsE[1])
plt.cla()
states = np.stack(traj)
plt.plot(states[:,0],label='x')
plt.plot(states[:,1],label='t')
plt.plot(states[:,2],label='dx')
plt.plot(states[:,3],label='dt')
plt.legend()

print(states.min(0))
print(states.max(0))

[ -1.381658     0.01388288  -6.304686   -13.619668  ]
[1.3114418 6.206531  5.9881225 8.445708 ]


In [ ]:
# old/debug